I am gonne use Deep Q learning for this project.CNN would be a lot better and algorithms with N step lookahead algorithms seems to work better than that i will maybe try them after i learn more about monte carlo tree search.

Most of good algorithms combine policy learning(really similar to Q learning) and tree search algorithms.

https://jonathan-hui.medium.com/alphago-how-it-works-technically-26ddcc085319

neural network code i will use later =codelateruse.py



In [1]:
from kaggle_environments import make, evaluate, utils
import numpy as np
import random
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

Loading environment lux_ai_2022 failed: No module named 'vec_noise'


2022-10-24 19:35:49.343112: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-24 19:35:49.452854: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-24 19:35:49.452866: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-24 19:35:49.472820: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-24 19:35:49.943174: W tensorflow/stream_executor/platform/de

<img src="./boardexample.png" alt="Drawing" style="width: 200px;"/>
obs.board property in this board will be

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 1, 2, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 2, 1, 2, 0, 2, 0]

I don't know where to start than i will copy another code with comments

In [2]:

class ConnectFourGym:

    def __init__(self, agent2="random"):
        ks_env = make('connectx', debug=True)
        self.env = ks_env.train([None, agent2])
        self.rows = ks_env.configuration.rows
        self.columns = ks_env.configuration.columns
        #We define actions possible here
        self.action_space = spaces.Discrete(self.columns)
        #We define all of space we are playing
        self.observation_space = spaces.Box(low=0,
                                            high=2,
                                            shape=(self.rows, self.columns, 1),
                                            dtype=int)
        # Tuple corresponding to the min and max possible rewards
        self.reward_range = (-10, 1)
        # StableBaselines throws error if these are not defined
        self.spec = None
        self.metadata = None

    def reset(self):
        self.obs = self.env.reset()
        return self.obs['board']

    def change_reward(self, old_reward, done):
        if old_reward == 1:
            return 1
        elif done:
            return -1
        else:
            return 0

    def step(self, agent):
        action = agent(self.obs)
        is_valid = (self.obs['board'][int(action)] == 0)
        if is_valid:
            self.obs, old_reward, done, _ = self.env.step(int(action))
            reward = self.change_reward(old_reward, done)
        else:
            reward, done, _ = -10, True, {}
        return [self.obs['board'],reward,done]

In [3]:
env = ConnectFourGym()

I am getting state action reward values with any agent

Next i will use q learning 
- I will use algorithm from this youtube video
- https://www.youtube.com/watch?v=wc-FxNENg9U

<img src="./loss.png" alt="Drawing"/>

Lets define constants for changing if necessary

In [ ]:
class DeepQNetwork(nn.Module):
    def __init__(self, lr,input_dims, fc1_dims, fc2_dims,n_actions):
        super(DeepQNetwork,self).__init__()
        self.input_dims = input_dims
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.n_actions = n_actions
        self.fc1 = nn.Linear(self.input_dims,self.fc1)